<a href="https://colab.research.google.com/github/MK316/Fall2023/blob/main/TranscriptionActivity01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌿 **Transcription practice activity** (asof 11/24)

[Shared file](https://docs.google.com/spreadsheets/d/12_hgfEpoYFlBbq-FHyMtpnv_bPuaxL-_c3jiyemffoU/edit?usp=drive_link)

# [1] Install packages

In [ ]:
!pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2

In [ ]:
%%capture
!pip install gtts jiwer matplotlib pandas

# [2] Google authentification & File to read

In [ ]:
#@markdown 🐳 Google authentification

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

# Authenticate and create the gspread client
creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Sheet and read it into a DataFrame
spreadsheet_name = 'transcription1124'  # Replace with your Google Sheet name
worksheet = gc.open(spreadsheet_name).sheet1

# Get all the values in the Sheet
rows = worksheet.get_all_values()

# Convert to a DataFrame
import pandas as pd
df = pd.DataFrame.from_records(rows[1:], columns=rows[0])

# Display the first few rows of the DataFrame
print(df['Group'].head())



[text to compare](https://raw.githubusercontent.com/MK316/Fall2023/main/original.md)

# [3] Calculating WER, finding missing and added words

WER is essentially a ratio of the total number of errors (substitutions, insertions, deletions) to the total number of words in the reference text.

Here is the basic formula for WER:

$$
WER = \frac{\text{Substitutions} + \text{Insertions} + \text{Deletions}}{\text{Number of Words in Reference}}
$$




In [ ]:
#@markdown 🐳 WER, Missing words, Added words

from jiwer import wer
import pandas as pd
from collections import Counter

# Function to calculate WER
def calculate_wer(recognized):
    return wer(original_text, recognized)

# Function to find missing words
def find_missing_words(recognized):
    original_words = original_text.split()
    recognized_words = recognized.split()

    original_count = Counter(original_words)
    recognized_count = Counter(recognized_words)

    missing_words = []
    for word in original_count:
        if original_count[word] > recognized_count.get(word, 0):
            missing_words.append(word)

    return ', '.join(missing_words)

# Function to find added words
def find_added_words(recognized):
    original_words = set(original_text.split())
    recognized_words = set(recognized.split())

    added_words = recognized_words - original_words
    return ', '.join(added_words)

# Prompt the user to enter the original text
original_text = input("Please enter the original text: ")

# Assuming df is your DataFrame and it has a column named 'Recognized'

# Apply the function to calculate WER for each row
df['WER'] = df['Recognized'].apply(calculate_wer)

# Apply the function to find missing words for each row
df['Missing'] = df['Recognized'].apply(find_missing_words)

# Apply the function to find added words for each row
df['Added'] = df['Recognized'].apply(find_added_words)

# Display the updated dataframe
# df
# Calculate and print the lowest WER
lowest_wer = df['WER'].min()
print(f"The lowest WER in the dataset is: {lowest_wer}")

# [4] Say the result

In [ ]:
#@markdown Say the result

import pandas as pd
from gtts import gTTS
import IPython.display as ipd

# Assuming df is your DataFrame with columns 'Group' and 'WER'

# Group the data by 'Group' and calculate the mean WER for each group
grouped_data = df.groupby('Group')['WER'].mean()

# Find the minimum WER value
min_wer = grouped_data.min()

# Find all groups that share the minimum WER
lowest_wer_groups = grouped_data[grouped_data == min_wer].index.tolist()

# Extract the numbers from the group names and format them
group_numbers = [int(''.join(filter(str.isdigit, group))) for group in lowest_wer_groups]
group_numbers = ', '.join(map(str, group_numbers[:-1])) + ' and ' + str(group_numbers[-1]) if len(group_numbers) > 1 else str(group_numbers[0])

# Create the message
message = f"Group number {group_numbers}? You did a great job for this activity!"

# Convert the message to speech
tts = gTTS(message)
tts.save("message.mp3")

# Play the message
ipd.Audio("message.mp3")


# [5] Plotting the result

In [ ]:
#@markdown Barplot (in order)
import matplotlib.pyplot as plt
import pandas as pd

# Assuming df is your DataFrame with columns 'Group' and 'WER'

# Group the data by 'Group' and calculate the mean WER for each group
grouped_data = df.groupby('Group')['WER'].mean()

# Sort the grouped data by WER values in ascending order
grouped_data_sorted = grouped_data.sort_values()

# Create the bar plot
plt.figure(figsize=(10, 6))
grouped_data_sorted.plot(kind='bar', color='skyblue')

# Set the y-axis limit
plt.ylim(0, 1)

# Adding titles and labels
plt.title('Word Error Rate by Group (Low to High)')
plt.xlabel('Group')
plt.ylabel('WER')

# Show the plot
plt.show()


In [ ]:
#@markdown Barplot all
import matplotlib.pyplot as plt
import pandas as pd

# Assuming df is your DataFrame with columns 'Group' and 'WER'

# Group the data by 'Group' and calculate the mean WER for each group
grouped_data = df.groupby('Group')['WER'].mean()

# Sort the grouped data by group names to maintain the original order (like G1, G2, ...)
grouped_data = grouped_data.sort_index()

# Create the bar plot
plt.figure(figsize=(10, 6))
grouped_data.plot(kind='bar', color='skyblue')

# Set the y-axis limit
plt.ylim(0, 1)

# Adding titles and labels
plt.title('Word Error Rate by Group')
plt.xlabel('Group')
plt.ylabel('WER')

# Show the plot
plt.show()


# [6] The result table

In [ ]:
df